# Watershed-based oversampling for imbalanced dataset classification: WSSMOTE

For several real-world problems, the dataset is composed of two or more imbalanced classes: a minority and majority ones. With usual machine learning methods, this imbalance often leads to poor results where the majority class is over-fitted while the minority class is misclassified. To alleviate these issues, several pre-processing methods, such as SMOTE andDBSMOTE, create new artificial points for the minority class. Neverthe-ess, these oversampling methods explicitly or implicitly make hypothesesabout the clusters size, shape, or density that may not fit the datasetin  practice.  We  propose  to  improve  these  oversampling methods and reduce cluster assumptions, by relying on a classifier:  the  watershed-cut.  We  called  this  method  WSSMOTE.

The following code generates four tabs containing G-mean score for several imabalanced datasets (https://sci2s.ugr.es/keel/datasets.php). These 12 tables corespond to Tab 3, 4 and 5 on the proposed article.

**Note**: To compile results faster, the following code is only 10-fold validation. Thus, results and more precisly approximations can be different from those you will find in the article. 

### Prerequises to launch file:

In [1]:
#pip install higra

In [2]:
#pip install smote-variants

In [3]:
#pip install -U imbalanced-learn

In [4]:
path = './Parameters.xlsx' #Change it with your own path

### Libraries to import

In [5]:
import numpy as np
import pandas as pd
import higra as hg
import scipy as sp
from sklearn import svm
from sklearn import tree
import sys

import math
import random
import wget
import imblearn

from zipfile import ZipFile

from sklearn.neighbors import NearestNeighbors

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from smote_variants import DBSMOTE
from imblearn.over_sampling import SMOTE
from gsmote import GeometricSMOTE

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

In [6]:
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning) #Ignore warning comments

### Import imbalanced datasets

#### Import zip file from KEEL datasets

In [7]:
# wget.download('https://sci2s.ugr.es/keel/dataset/data/imbalanced/wisconsin.zip')
# wget.download('https://sci2s.ugr.es/keel/dataset/data/imbalanced/yeast4.zip')
# wget.download('https://sci2s.ugr.es/keel/dataset/data/imbalanced/vehicle1.zip')
# wget.download('https://sci2s.ugr.es/keel/keel-dataset/datasets/imbalanced/imb_noisyBordExamples/03subcl5-600-5-70-BI.zip')
# wget.download('https://sci2s.ugr.es/keel/keel-dataset/datasets/imbalanced/imb_noisyBordExamples/paw02a-800-7-60-BI.zip')
# wget.download('https://sci2s.ugr.es/keel/dataset/data/imbalanced/ecoli1.zip')
# wget.download('https://sci2s.ugr.es/keel/dataset/data/imbalanced/segment0.zip')
# wget.download('https://sci2s.ugr.es/keel/dataset/data/imbalanced/pima.zip')
# wget.download('https://sci2s.ugr.es/keel/dataset/data/imbalanced/cleveland-0_vs_4.zip')
# wget.download('https://sci2s.ugr.es/keel/dataset/data/imbalanced/yeast6.zip')
# wget.download('https://sci2s.ugr.es/keel/dataset/data/imbalanced/page-blocks-1-3_vs_4.zip')
# wget.download('https://sci2s.ugr.es/keel/dataset/data/imbalanced/glass-0-1-5_vs_2.zip')

#### Transform zip file into dat file into arrays contening data and labels

In [8]:
def examples_transform(examples_zip, examples_data):
    examples = [] #containing n dataframe with n the number of example
    labels = [[0] for i in range(len(examples_zip))] # contains n label array with n the number of example

    for ind_ex in range(len(examples_zip)):
        zip = ZipFile(examples_zip[ind_ex])
        zip.extractall() 

        #Open .dat files and compile data and labels information into dataframe
        file = open(examples_data[ind_ex])
        data = file.read()

        data = data.split('@')[-1]
        data = data.split('\n')[1:]
        data = np.array(data)
        data_f = []
        for index in range(len(data)):
            data_f.append(data[index].split(','))
        data_f = data_f[:-1]
        data_f = pd.DataFrame(data_f)
        
        nb = data_f.shape[1] - 1 #last columns = labels
        data_f[nb] = [data_f[nb][i].strip() for i in range(len(data_f[nb]))]
        elment = np.unique(data_f[nb])
        data_f[nb].replace({elment[0]: '1', elment[1]: '2'}, inplace=True)
        
        Y = np.array(data_f[nb], dtype=int)

        X = data_f.drop(columns=nb)
        X = X.astype(float)
        X = np.array(X)
        
        examples.append(X)
        labels[ind_ex] = Y
        file.close()
    return examples, labels

In [9]:
#Execution of examples function with imbalanced datasets studied in the paper
examples_zip = ['wisconsin.zip', 'yeast4.zip',  'vehicle1.zip', 
            '03subcl5-600-5-70-BI.zip', 'paw02a-800-7-60-BI.zip', 'ecoli1.zip', 'pima.zip', 'yeast6.zip', 
               'page-blocks-1-3_vs_4.zip', 'glass-0-1-5_vs_2.zip']
examples_data = ['wisconsin.dat', 'yeast4.dat',  'vehicle1.dat', 
            '03subcl5-600-5-70-BI.dat', 'paw02a-800-7-60-BI.dat', 'ecoli1.dat', 'pima.dat', 'yeast6.dat', 
               'page-blocks-1-3_vs_4.dat', 'glass-0-1-5_vs_2.dat' ]

examples, labels = examples_transform(examples_zip, examples_data)

## WSSMOTE algorithm

In [10]:
def dijkstra(graph, source):
    '''
    Dijkstra algorithm
    Input:
        - graph -- containing all the distance between data points 
        - source -- a specific data points 
    Output:
        - 
    '''
    distances, parents = dict(), dict()

    for node in graph:
        if node not in distances.keys():
            distances[node] = float('Inf')
        if node not in parents.keys():
            parents[node] = None
    distances[source] = 0

    for ind_graph in range(len(graph)-1):
        for pt_i in graph:
            for pt_j in graph[pt_i]:
                assert graph[pt_i][pt_j] >= 0
                if distances[pt_j] > distances[pt_i] + graph[pt_i][pt_j]:
                    distances[pt_j] = distances[pt_i] + graph[pt_i][pt_j]
                    parents[pt_j] = pt_i

    return distances, parents

In [11]:
def higra_clustering(X_min, n_nei):
    '''
    Generate clusters using Higra
    Input:
        - X_min: minority dataset 
        - n_nei: Number of neighbors 
    Output:
        - labels: clustering labelling
        - num_labels: numbers of clusters
    '''
    graph, edge_weights = hg.make_graph_from_points(X_min, graph_type='knn', mode='distance', n_neighbors=n_nei)
    labels = hg.labelisation_watershed(graph, edge_weights)
    num_labels = np.max(labels)
    return labels - 1, num_labels

In [12]:
def WSSMOTE(X, labels, n_nei, coeff_min, n_add):
    '''
    WSSMOTE algorithm
    Input:
        - X: data points
        - labels: array
        - n_nei: number of neighbors
        - coeff_min: label of the minority class
    Output:
        - np.vstack([X, samples]): oversampling data 
        - np.hstack([labels, np.repeat(coeff_min, len(samples))]): oversampling labels
    '''
     #Compute clusters
    X_min = X[labels == coeff_min]
    labels_clust, num_labels = higra_clustering(X_min, n_nei)

    #Construct array of clusters
    clusters = [np.where(labels_clust == ind_label)[0] for ind_label in range(num_labels)]
    cluster_sizes = np.array([np.sum(labels_clust == i) for i in range(num_labels)])
    cluster_dist = cluster_sizes/(np.sum(cluster_sizes))

    #Graphs, shortest path and centroids
    graphs, centroid_indices, sorthest_path = [], [], []
    
    for num_label in range(num_labels):
        cluster = X_min[clusters[num_label]]
        nn = NearestNeighbors(n_neighbors=len(cluster), metric='euclidean', n_jobs=1).fit(cluster)
        centroid_ind = nn.kneighbors([np.mean(cluster, axis=0)])[1][0][0] # closest data of the real centroid
        centroid_indices.append(centroid_ind)
        
        graph = dict() # distance between two points in cluster 
        
        dist, ind = nn.kneighbors(cluster)
#         dist = dist[:, 1:]
#         ind = ind[:, 1:]
        for pt_cluster_i in range(len(cluster)):
            graph[pt_cluster_i] = dict()
            for pt_cluster_j in range(len(cluster)):
                graph[pt_cluster_i][pt_cluster_j] = dist[pt_cluster_i][ind[pt_cluster_i] == pt_cluster_j][0]
                    
        sorthest_path.append(dijkstra(graph, centroid_ind))
        graphs.append(graph)
        
    #New data points creation
    samples = []
    while len(samples) < n_add*abs(len(np.where(labels==coeff_min)[0]) - len(np.where(labels==((1+coeff_min)%2))[0])):
        cluster_idx = np.random.choice(np.arange(len(clusters)), p=cluster_dist)
        cluster = X_min[clusters[cluster_idx]]
        idx = np.random.choice(range(len(clusters[cluster_idx])))

        distances, parents = sorthest_path[cluster_idx]

        path = [idx]
        while not parents[path[-1]] is None:
            path.append(parents[path[-1]])

        if len(path) == 1:
            X_b = cluster[path[0]]
            samples.append(X_b)
        else:
            X_a = cluster[path[0]]
            X_b = cluster[path[-1]]
            sample = X_a + (X_b-X_a)*np.random.uniform(0,1)
            samples.append(sample)
    return np.vstack([X, samples]), np.hstack([labels, np.repeat(coeff_min, len(samples))])

## Classifers algorithms

In [13]:
def score(res, y_test):
    '''
    Gmean score
    Input: 
        - Res: predicted labels
        - y_test: real labels
    Output:
        -Gmean accuracy
    '''
    array = confusion_matrix(y_test, res)
    if len(array[0]) == 2:
        tn, fp, fn, tp = array.ravel()
        if (tn*tp) == 0:
            acc = 0
        else:
            acc = math.sqrt((tp*tn)/((tp+fn)*(fp+tn)))
    else: 
        acc =0
    return acc

In [14]:
def watershed_cut(X_train, X_test, y_train, y_test):
    '''
    Watershed cut
    Input:
        - X_train
        - X_test
        - y_train
        - y_test
    Output:
        - y_keep = labels predicted
    '''
    X_tot = np.concatenate((X_train, X_test))
    seeds = np.int_(np.concatenate((y_train, np.zeros(len(y_test))))) # concatenate y_train and 0 for the test

    best_acc = 0
    
    for n_nei in np.arange(5,66,10): #to find the best number of neighbors
        graph, edge_weights = hg.make_graph_from_points(X_tot, graph_type='knn', mode='distance', 
                                                        n_neighbors=n_nei, metric='euclidian')
        y_watershed = hg.labelisation_seeded_watershed(graph, edge_weights, seeds)
        y_watershed_test = y_watershed[len(y_train):]

        if len(np.unique(y_watershed_test)) <= 2:
            acc = score(y_watershed_test, y_test)
        else:
            acc = 0
        if acc >= best_acc:
            best_acc = acc 
            y_keep = y_watershed_test
    return y_keep

In [15]:
def SVM_method(X_train, X_test, y_train, y_test):
    '''
    SVM classification algorithm
    Input:
        - X_train
        - X_test
        - y_train
        - y_test
    Output:
        - y_SVM = labels predicted
    '''
    clf = svm.LinearSVC()
    clf = clf.fit(X_train, y_train)
    y_SVM = clf.predict(X_test)
    return y_SVM

In [16]:
def DecisionTree_method(X_train, X_test, y_train, y_test):
    '''
    Decision Tree algorithm
    Input:
        - X_train
        - X_test
        - y_train
        - y_test
    Output:
        - y_tree = labels predicted
    '''
    clf = tree.DecisionTreeClassifier()
    clf =  clf.fit(X_train, y_train)
    y_tree = clf.predict(X_test)
    return y_tree

In [17]:
def KNeiClass_method(X_train, X_test, y_train, y_test):
    '''
    KNN algorithm
    Input:
        - X_train
        - X_test
        - y_train
        - y_test
    Output:
        - y_keep = labels predicted
    '''
    best_acc = 0
    for n_nei in np.arange(2,55,10):  #to find the best number of neighbors
        clf = KNeighborsClassifier(n_neighbors=n_nei)
        clf = clf.fit(X_train, y_train)
        y_nei = clf.predict(X_test)
        array = confusion_matrix(y_test, y_nei)
        acc = score(y_nei, y_test)
        if acc >= best_acc:
            best_acc = acc
            y_keep = y_nei
    return y_keep

## Compile best parameters for each over sampling, classifier algorithms and imbalanced datasets

In [18]:
SMOTE_param = pd.read_excel(path, 'SMOTE') #Compute SMOTE parameter
SMOTE_param.index = SMOTE_param['Unnamed: 0']
SMOTE_param = SMOTE_param.drop(columns='Unnamed: 0')
SMOTE_param = SMOTE_param.to_dict()

DBSMOTE_eps = pd.read_excel(path, 'DBSMOTE-eps') #Compute DBSMOTE parameter
DBSMOTE_eps.index = DBSMOTE_eps['Unnamed: 0']
DBSMOTE_eps = DBSMOTE_eps.drop(columns='Unnamed: 0')
DBSMOTE_eps = DBSMOTE_eps.to_dict()

DBSMOTE_min = pd.read_excel(path, 'DBSMOTE-min')
DBSMOTE_min.index = DBSMOTE_min['Unnamed: 0']
DBSMOTE_min = DBSMOTE_min.drop(columns='Unnamed: 0')
DBSMOTE_min = DBSMOTE_min.to_dict()

DBSMOTE_param_ = [DBSMOTE_eps, DBSMOTE_min]
DBSMOTE_param = {}
for key in DBSMOTE_eps.keys():
    for k in DBSMOTE_eps[key].keys():
        if not key in DBSMOTE_param.keys():
            DBSMOTE_param[key] = dict()
        DBSMOTE_param[key][k] = tuple(d[key][k] for d in DBSMOTE_param_)

WSMOTE_nn = pd.read_excel(path, 'WSSMOTE-nn') #Compute WSSMOTE parameter
WSMOTE_nn.index = WSMOTE_nn['Unnamed: 0']
WSMOTE_nn = WSMOTE_nn.drop(columns='Unnamed: 0')
WSMOTE_nn = WSMOTE_nn.to_dict()

WSMOTE_add = pd.read_excel(path, 'WSSMOTE-n_add')
WSMOTE_add.index = WSMOTE_add['Unnamed: 0']
WSMOTE_add = WSMOTE_add.drop(columns='Unnamed: 0')
WSMOTE_add = WSMOTE_add.to_dict()

WSSMOTE_param_ = [WSMOTE_nn, WSMOTE_add]
WSSMOTE_param = {}
for key in WSMOTE_nn.keys():
    for k in WSMOTE_nn[key].keys():
        if not key in WSSMOTE_param.keys():
            WSSMOTE_param[key] = dict()
        WSSMOTE_param[key][k] = tuple(d[key][k] for d in WSSMOTE_param_)
    
    
GSMOTE_nn = pd.read_excel(path, 'GSMOTE-nn') #Compute WSSMOTE parameter
GSMOTE_nn.index = GSMOTE_nn['Unnamed: 0']
GSMOTE_nn = GSMOTE_nn.drop(columns='Unnamed: 0')
GSMOTE_nn = GSMOTE_nn.to_dict()

In [19]:
dicts_paramater = dict() # Dictionnary: contains all parameters 
dicts_paramater['SMOTE'] = SMOTE_param
dicts_paramater['WSSMOTE'] = WSSMOTE_param
dicts_paramater['DBSMOTE'] = DBSMOTE_param
dicts_paramater['GSMOTE'] = GSMOTE_nn

# G-means Score

In [20]:
def generate_dico_results(classifiers, classifiers_name, dicts_paramater, oversampling_methods, examples_name, coeffs_min):
    '''
    Generate three dictionnary containing Gmean, F1 or AUC score for each imbalanced datasets
    Input:
        - classifiers: list of classifier algortihm used
        - classifiers_name: list of classifier algoritm names used
        - dict_paramters: dictionnary containing best parameters for oversampling method
        - oversampling_method: list of oversampling methods used
        - coeffs_min: list of the label of the minority data points for each imbalanced datasets
    Output:
        - dictionnary - keys = oversampling method - values: scores
    '''
    dico_over_med_g_mean, dico_over_std_g_mean = dict(), dict()
    dico_over_med_auc, dico_over_std_auc = dict(), dict()
    dico_over_med_f, dico_over_std_f= dict(), dict()
    
    for over_method in oversampling_methods:
        tab_med_g_mean, tab_std_g_mean = pd.DataFrame(), pd.DataFrame()
        tab_med_auc, tab_std_auc = pd.DataFrame(), pd.DataFrame()
        tab_med_f, tab_std_f = pd.DataFrame(), pd.DataFrame()
        
        for idx, example in enumerate(examples):
            print(examples_name[idx])
            X, y = example, labels[idx]
            data_median_g_mean, data_std_g_mean = [], []
            data_median_auc, data_std_auc = [], []
            data_median_f, data_std_f = [], []
            for ind_class, classifier in enumerate(classifiers):
                acc_g_mean, acc_f, acc_auc = [], [], []
                for num_round in range(10):
                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
                    if over_method == 'without':
                        X_train_over, y_train_over = X_train.copy(), y_train.copy()
                    elif over_method == 'SMOTE':
                        sm = SMOTE(k_neighbors = dicts_paramater[over_method][examples_name[idx]][classifiers_name[ind_class]])
                        X_train_over, y_train_over = sm.fit_resample(X_train, y_train)
                    elif over_method == 'GSSMOTE':
                        sm = GeometricSMOTE(k_neighbors=dicts_paramater['GSMOTE'][examples_name[idx]][classifiers_name[ind_class]])
                        X_train_over, y_train_over = sm.fit_resample(X_train, y_train)
                    elif over_method == 'DBSMOTE':
                        sm = DBSMOTE(eps = dicts_paramater[over_method][examples_name[idx]][classifiers_name[ind_class]][0], 
                                 min_samples= dicts_paramater[over_method][examples_name[idx]][classifiers_name[ind_class]][1])
                        X_train_over, y_train_over = sm.fit_resample(X_train, y_train)
                    elif over_method == 'WSSMOTE':
                        X_train_over, y_train_over= WSSMOTE(X_train, y_train, n_nei=dicts_paramater[over_method][examples_name[idx]][classifiers_name[ind_class]][0],
                                                        coeff_min=coeffs_min[idx], n_add =dicts_paramater[over_method][examples_name[idx]][classifiers_name[ind_class]][1])
                    res = classifier(X_train_over, X_test, y_train_over, y_test)
                    acc_g_mean.append(score(res, y_test))
                    acc_f.append(f1_score(y_test, res))
                    try:
                        acc_auc.append(roc_auc_score(y_test, res))
                    except ValueError:
                        acc_auc.append(0)
                    
                    
                data_median_g_mean.append(round(np.median(acc_g_mean)*100, 2))
                data_std_g_mean.append(round(sp.stats.iqr(acc_g_mean)*100, 2))
                data_median_f.append(round(np.median(acc_f)*100, 2))
                data_std_f.append(round(sp.stats.iqr(acc_f)*100, 2))
                data_median_auc.append(round(np.median(acc_auc)*100, 2))
                data_std_auc.append(round(sp.stats.iqr(acc_auc)*100, 2))
                
            tab_med_g_mean[examples_name[idx]], tab_std_g_mean[examples_name[idx]]  = data_median_g_mean, data_std_g_mean
            tab_med_f[examples_name[idx]], tab_std_f[examples_name[idx]]  = data_median_f, data_std_f
            tab_med_auc[examples_name[idx]], tab_std_auc[examples_name[idx]]  = data_median_auc, data_std_auc
            
        tab_med_g_mean.index, tab_std_g_mean.index = classifiers_name, classifiers_name
        tab_med_f.index, tab_std_f.index = classifiers_name, classifiers_name
        tab_med_auc.index, tab_std_auc.index = classifiers_name, classifiers_name
        
        dico_over_med_g_mean[over_method], dico_over_std_g_mean[over_method] = tab_med_g_mean, tab_std_g_mean
        dico_over_med_f[over_method], dico_over_std_f[over_method] = tab_med_f, tab_std_f
        dico_over_med_auc[over_method], dico_over_std_auc[over_method] = tab_med_auc, tab_std_auc
        
    return (dico_over_med_g_mean, dico_over_std_g_mean), (dico_over_med_f, dico_over_std_f), (dico_over_med_auc, dico_over_std_auc)

In [21]:
classifiers =  [watershed_cut, KNeiClass_method, DecisionTree_method, SVM_method]
classifiers_name = ['WC','KNN', 'DT', 'SVM']
oversampling_methods = ['without', 'SMOTE', 'DBSMOTE', 'GSSMOTE', 'WSSMOTE']
examples_name = ['wisconsin', 'yeast4', 'vehicle1', 'Subcl35', 'Paw', 'ecoli1', 'pima', 'yeast6', 'page_blocks', 'glass']
coeffs_min = [2, 2, 2, 1, 1, 2, 2, 2, 2, 2]

score_g_mean, score_f, score_auc = generate_dico_results(classifiers, classifiers_name, dicts_paramater, oversampling_methods, examples_name, coeffs_min)

wisconsin


C:\Users\oucht\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\oucht\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\oucht\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\oucht\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\oucht\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

yeast4
vehicle1


C:\Users\oucht\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\oucht\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\oucht\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\oucht\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\oucht\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

Subcl35


...

C:\Users\oucht\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\oucht\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\oucht\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\oucht\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\oucht\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

yeast6
page_blocks


C:\Users\oucht\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\oucht\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\oucht\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\oucht\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\oucht\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

glass


C:\Users\oucht\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\oucht\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\oucht\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\oucht\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\oucht\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

### Results

##### G-Mean

In [22]:
dico_over_med, dico_over_std = score_g_mean[0], score_g_mean[1]

In [23]:
print('Tab oversampling method: without' )
print(pd.DataFrame.from_dict(dico_over_med['without']))
print(pd.DataFrame.from_dict(dico_over_std['without']))

Tab oversampling method: without
     wisconsin  yeast4  vehicle1  Subcl35    Paw  ecoli1   pima  yeast6  \
WC       95.34   59.99     60.80    61.52  68.43   81.74  63.57   71.60   
KNN      96.61   31.57     52.96    71.89  74.35   85.18  66.54   65.90   
DT       94.52   56.72     65.11    63.48  67.47   84.09  65.53   67.91   
SVM      96.96    0.00     53.82     0.00   0.00   83.43  28.22    0.00   

     page_blocks  glass  
WC         89.19  48.77  
KNN        79.68   0.00  
DT         99.45  56.82  
SVM        76.09   0.00  
     wisconsin  yeast4  vehicle1  Subcl35    Paw  ecoli1   pima  yeast6  \
WC        3.89   21.11      7.10    10.50   9.64    8.84   5.34   15.00   
KNN       2.19   35.36      6.72     5.73   8.31    5.39   5.32   16.82   
DT        3.52   13.95      6.03    12.14   6.98    8.28   6.49   18.05   
SVM       2.20    0.00     31.07    41.46  25.61    8.36  27.13    0.00   

     page_blocks  glass  
WC         15.33  69.32  
KNN        15.41   0.00  
DT     

In [24]:
print('Tab oversampling method: SMOTE' )
print(pd.DataFrame.from_dict(dico_over_med['SMOTE']))
print(pd.DataFrame.from_dict(dico_over_std['SMOTE']))

Tab oversampling method: SMOTE
     wisconsin  yeast4  vehicle1  Subcl35    Paw  ecoli1   pima  yeast6  \
WC       95.52   76.53     67.57    70.73  78.04   86.17  64.20   78.18   
KNN      97.83   88.49     71.09    81.14  85.94   91.10  73.67   90.02   
DT       93.42   67.04     67.77    73.38  76.34   86.63  64.97   69.29   
SVM      97.20   82.24     67.43    37.71  28.76   86.92  32.58   87.77   

     page_blocks  glass  
WC         99.43  61.72  
KNN        98.88  74.44  
DT         99.46  56.83  
SVM        95.84  20.97  
     wisconsin  yeast4  vehicle1  Subcl35    Paw  ecoli1   pima  yeast6  \
WC        2.88    9.50      3.94     7.37   8.95    6.86   4.28   18.07   
KNN       2.22    6.89      4.85     4.60   3.52    2.29   4.53   11.83   
DT        3.08   12.83      5.97     8.54   7.02    8.77   4.50   11.93   
SVM       2.12    6.90     15.98    24.04  42.74    5.21  37.23    6.20   

     page_blocks  glass  
WC          1.15  16.48  
KNN         3.77  17.14  
DT       

In [25]:
print('Tab oversampling method: DBSMOTE' )
print(pd.DataFrame.from_dict(dico_over_med['DBSMOTE']))
print(pd.DataFrame.from_dict(dico_over_std['DBSMOTE']))

Tab oversampling method: DBSMOTE
     wisconsin  yeast4  vehicle1  Subcl35    Paw  ecoli1   pima  yeast6  \
WC       95.08   53.13     60.06    66.65  73.20   84.24  62.97   75.14   
KNN      96.04   77.35     52.91    76.61  85.15   86.75  73.55   87.86   
DT       94.22   54.48     65.90    67.55  70.40   82.73  67.25   69.97   
SVM      96.48   71.96     57.96    32.55  33.73   85.24  42.03   89.84   

     page_blocks  glass  
WC         92.05  61.64  
KNN        81.65  73.54  
DT         99.44  55.81  
SVM        95.67   0.00  
     wisconsin  yeast4  vehicle1  Subcl35    Paw  ecoli1   pima  yeast6  \
WC        2.43   21.96      6.88    11.53   7.29   13.27   6.45   15.64   
KNN       2.66   11.34      5.07     4.57   4.85    5.06   4.31   11.05   
DT        2.16   11.24      4.93     7.41   7.82    6.04   3.44   13.75   
SVM       2.59   10.00     45.56    45.43  47.87    7.08  34.20   13.59   

     page_blocks  glass  
WC         12.70  26.64  
KNN        18.73  17.57  
DT     

In [26]:
print('Tab oversampling method: GSSMOTE' )
print(pd.DataFrame.from_dict(dico_over_med['GSSMOTE']))
print(pd.DataFrame.from_dict(dico_over_std['GSSMOTE']))

Tab oversampling method: GSSMOTE
     wisconsin  yeast4  vehicle1  Subcl35    Paw  ecoli1   pima  yeast6  \
WC       95.45   72.39     62.10    69.40  74.99   82.16  64.19   79.59   
KNN      97.59   85.75     71.53    78.21  84.70   90.58  71.85   88.07   
DT       94.37   57.12     66.92    72.67  74.26   84.52  68.52   70.23   
SVM      96.70   82.17     45.33    41.42  39.16   86.09  40.35   87.94   

     page_blocks  glass  
WC         93.09  64.92  
KNN        95.35  77.81  
DT         94.22  50.28  
SVM        95.45   0.00  
     wisconsin  yeast4  vehicle1  Subcl35    Paw  ecoli1   pima  yeast6  \
WC        3.09   14.50      5.61     4.75   7.27    7.58   4.40    6.71   
KNN       1.67    6.35      5.58     4.60   2.87    4.05   4.27    3.52   
DT        4.01   20.80      5.25     8.63  10.64    6.57   3.27   17.57   
SVM       2.23    9.01     39.20    13.15  26.85    4.45  37.41   11.40   

     page_blocks  glass  
WC          9.91  36.36  
KNN         4.43  12.05  
DT     

In [27]:
print('Tab oversampling method: WSSMOTE' )
print(pd.DataFrame.from_dict(dico_over_med['WSSMOTE']))
print(pd.DataFrame.from_dict(dico_over_std['WSSMOTE']))

Tab oversampling method: WSSMOTE
     wisconsin  yeast4  vehicle1  Subcl35    Paw  ecoli1   pima  yeast6  \
WC       95.95   61.71     62.40    63.94  72.62   85.06  63.64   74.87   
KNN      98.02   87.40     70.02    76.03  85.71   91.53  69.62   88.74   
DT       94.00   59.07     69.46    66.88  70.87   85.16  67.15   70.11   
SVM      96.94   71.22     71.87    42.86   0.00   88.45  49.61   86.62   

     page_blocks  glass  
WC         92.32  57.88  
KNN        97.66  78.08  
DT        100.00  47.94  
SVM        95.45  30.63  
     wisconsin  yeast4  vehicle1  Subcl35    Paw  ecoli1   pima  yeast6  \
WC        1.93   12.98      6.17    14.56  10.33   10.43   5.11   10.90   
KNN       1.51    5.05      5.00     5.33   4.38    3.96   3.56    6.81   
DT        2.37   10.21      5.20     7.88   9.87    7.88   5.49   17.62   
SVM       1.75    3.36     17.15    15.48  37.18    4.75  15.54    7.90   

     page_blocks  glass  
WC         12.43  17.72  
KNN         3.89  10.23  
DT     

##### F1-Score 

In [28]:
dico_over_med, dico_over_std = score_f[0], score_f[1]

In [29]:
print('Tab oversampling method: without' )
print(pd.DataFrame.from_dict(dico_over_med['without']))
print(pd.DataFrame.from_dict(dico_over_std['without']))

Tab oversampling method: without
     wisconsin  yeast4  vehicle1  Subcl35    Paw  ecoli1   pima  yeast6  \
WC       97.13   97.82     81.53    41.64  50.64   92.23  74.81   98.71   
KNN      97.78   98.45     85.03    50.85  50.49   93.27  82.44   99.06   
DT       96.24   97.38     82.40    47.93  48.78   92.83  77.36   98.37   
SVM      97.86   98.29     85.49     0.00   0.00   93.64  76.74   98.81   

     page_blocks  glass  
WC         98.88  92.06  
KNN        98.36  94.66  
DT         99.45  92.19  
SVM        97.22  93.94  
     wisconsin  yeast4  vehicle1  Subcl35    Paw  ecoli1   pima  yeast6  \
WC        2.21    1.04      3.86    10.52  13.71    3.61   5.20    0.48   
KNN       1.58    0.69      3.57     5.30  10.61    2.89   2.57    0.48   
DT        1.83    0.85      4.27    14.39  13.73    3.61   3.88    0.82   
SVM       1.26    0.52      4.50    26.26  14.76    3.22  42.63    0.52   

     page_blocks  glass  
WC          1.54   3.94  
KNN         1.51   3.12  
DT     

In [30]:
print('Tab oversampling method: SMOTE' )
print(pd.DataFrame.from_dict(dico_over_med['SMOTE']))
print(pd.DataFrame.from_dict(dico_over_std['SMOTE']))

Tab oversampling method: SMOTE
     wisconsin  yeast4  vehicle1  Subcl35    Paw  ecoli1   pima  yeast6  \
WC       97.01   95.53     78.92    49.97  55.71   92.61  74.74   97.19   
KNN      98.30   92.75     73.93    55.28  53.26   92.21  75.22   95.73   
DT       95.90   96.64     83.60    53.61  50.45   91.37  75.25   97.55   
SVM      98.10   92.94     82.88    25.29  19.14   90.46  76.11   95.36   

     page_blocks  glass  
WC         99.44  88.14  
KNN        98.87  84.41  
DT         99.46  92.06  
SVM        96.87  19.93  
     wisconsin  yeast4  vehicle1  Subcl35    Paw  ecoli1   pima  yeast6  \
WC        1.87    1.69      4.18    14.38  10.07    4.18   4.59    0.74   
KNN       1.57    1.59      5.10     6.85   7.36    3.42   4.64    1.18   
DT        1.98    1.18      2.12     8.94  12.65    3.03   5.78    0.48   
SVM       1.73    1.26     15.01    22.76  25.88    3.60  58.20    1.28   

     page_blocks  glass  
WC          1.11   6.02  
KNN         3.44  15.56  
DT       

In [31]:
print('Tab oversampling method: DBSMOTE' )
print(pd.DataFrame.from_dict(dico_over_med['DBSMOTE']))
print(pd.DataFrame.from_dict(dico_over_std['DBSMOTE']))

Tab oversampling method: DBSMOTE
     wisconsin  yeast4  vehicle1  Subcl35    Paw  ecoli1   pima  yeast6  \
WC       96.67   97.73     80.32    45.64  52.79   92.38  75.00   98.53   
KNN      97.42   95.22     84.40    51.42  53.25   92.47  76.46   98.70   
DT       96.24   97.20     83.17    49.53  50.00   92.31  77.30   98.61   
SVM      97.67   94.57     84.79    11.15  22.23   92.23  77.70   96.79   

     page_blocks  glass  
WC         98.86  93.65  
KNN        98.89  84.62  
DT         99.44  92.19  
SVM        97.74  92.31  
     wisconsin  yeast4  vehicle1  Subcl35    Paw  ecoli1   pima  yeast6  \
WC        1.47    1.00      3.58     9.48  13.18    3.17   4.40    0.65   
KNN       1.49    2.28      2.35    10.04  11.27    3.84   5.05    0.89   
DT        1.68    1.13      3.44    12.32   9.23    3.14   3.84    0.65   
SVM       1.69    2.88      4.97    26.61  26.60    2.88  19.30    0.66   

     page_blocks  glass  
WC          1.11   3.58  
KNN         1.52  14.02  
DT     

In [32]:
print('Tab oversampling method: GSSMOTE' )
print(pd.DataFrame.from_dict(dico_over_med['GSSMOTE']))
print(pd.DataFrame.from_dict(dico_over_std['GSSMOTE']))

Tab oversampling method: GSSMOTE
     wisconsin  yeast4  vehicle1  Subcl35    Paw  ecoli1   pima  yeast6  \
WC       96.64   95.50     80.16    49.56  52.64   90.29  73.32   97.02   
KNN      98.31   92.51     76.75    50.61  51.32   92.31  77.82   95.07   
DT       96.19   97.20     81.95    47.32  49.49   92.78  76.92   98.01   
SVM      97.49   91.67     82.61    24.53  22.78   90.55  68.61   94.19   

     page_blocks  glass  
WC         98.28  89.76  
KNN        95.27  83.64  
DT         99.44  90.47  
SVM        95.40  88.89  
     wisconsin  yeast4  vehicle1  Subcl35    Paw  ecoli1   pima  yeast6  \
WC        1.92    1.15      2.56    10.01   8.69    4.31   3.03    0.91   
KNN       1.58    1.83      5.39     9.03   5.17    3.17   3.43    2.32   
DT        2.27    0.56      1.58     9.15   7.69    3.03   2.32    0.82   
SVM       1.70    1.61     21.75    15.43  12.25    3.18  46.99    1.92   

     page_blocks  glass  
WC          1.14   6.30  
KNN         4.07   9.27  
DT     

In [33]:
print('Tab oversampling method: WSSMOTE' )
print(pd.DataFrame.from_dict(dico_over_med['WSSMOTE']))
print(pd.DataFrame.from_dict(dico_over_std['WSSMOTE']))

Tab oversampling method: WSSMOTE
     wisconsin  yeast4  vehicle1  Subcl35    Paw  ecoli1   pima  yeast6  \
WC       97.21   97.28     78.01    44.95  52.19   92.85  72.40   98.26   
KNN      98.27   89.84     69.82    50.37  56.03   92.39  71.00   95.62   
DT       95.88   97.02     83.40    48.05  52.51   93.33  75.37   98.19   
SVM      97.62   73.29     78.50    27.11   0.00   90.35  62.87   92.05   

     page_blocks  glass  
WC         98.88  90.62  
KNN        97.63  81.92  
DT        100.00  91.79  
SVM        96.65  17.65  
     wisconsin  yeast4  vehicle1  Subcl35    Paw  ecoli1   pima  yeast6  \
WC        1.25    0.83      2.68     9.63  12.03    3.09   3.01    0.83   
KNN       1.30    2.81      4.43     9.48  10.77    3.84   4.94    1.42   
DT        0.95    0.78      1.87    11.20  11.42    4.23   3.98    0.66   
SVM       1.33    4.53     15.04    12.71  21.70    3.19  37.30    2.10   

     page_blocks  glass  
WC          1.16   3.17  
KNN         4.00  16.51  
DT     

#### AUC-score

In [34]:
dico_over_med, dico_over_std = score_auc[0], score_auc[1]

In [35]:
print('Tab oversampling method: without' )
print(pd.DataFrame.from_dict(dico_over_med['without']))
print(pd.DataFrame.from_dict(dico_over_std['without']))

Tab oversampling method: without
     wisconsin  yeast4  vehicle1  Subcl35    Paw  ecoli1   pima  yeast6  \
WC       95.41   67.66     63.35    65.76  71.83   82.46  64.49   75.35   
KNN      96.64   54.83     61.18    72.81  75.57   85.83  69.60   71.57   
DT       94.55   64.93     66.16    67.84  71.00   84.57  66.90   72.22   
SVM      96.99   50.00     61.94    50.00  50.00   83.77  53.35   50.00   

     page_blocks  glass  
WC         89.72  59.27  
KNN        81.46  50.00  
DT         99.45  65.10  
SVM        78.73  50.00  
     wisconsin  yeast4  vehicle1  Subcl35   Paw  ecoli1  pima  yeast6  \
WC        3.81   12.06      5.79     7.42  5.96    7.71  5.61   11.86   
KNN       2.16    6.25      3.75     4.76  7.10    5.07  4.17   11.16   
DT        3.33    7.77      4.36     8.80  4.70    7.72  5.51   12.37   
SVM       2.22    0.00     16.77     0.88  1.26    7.04  6.25    0.00   

     page_blocks  glass  
WC         13.75  26.02  
KNN        11.94   0.00  
DT          7.34 

In [36]:
print('Tab oversampling method: SMOTE' )
print(pd.DataFrame.from_dict(dico_over_med['SMOTE']))
print(pd.DataFrame.from_dict(dico_over_std['SMOTE']))

Tab oversampling method: SMOTE
     wisconsin  yeast4  vehicle1  Subcl35    Paw  ecoli1   pima  yeast6  \
WC       95.55   77.76     67.93    71.95  78.66   86.21  65.56   79.64   
KNN      97.84   88.50     71.89    81.43  86.56   91.25  73.85   90.03   
DT       93.46   71.39     69.30    73.83  76.76   86.87  65.54   73.01   
SVM      97.22   82.34     71.02    50.22  53.80   87.09  53.88   87.89   

     page_blocks  glass  
WC         99.44  64.73  
KNN        98.88  74.86  
DT         99.46  64.41  
SVM        95.93  52.25  
     wisconsin  yeast4  vehicle1  Subcl35   Paw  ecoli1   pima  yeast6  \
WC        2.86    7.90      3.64     6.93  7.01    6.75   4.46   14.89   
KNN       2.22    7.15      4.32     4.78  3.83    2.49   4.69   11.47   
DT        2.90    9.67      4.50     7.28  6.22    8.08   4.16    8.51   
SVM       2.10    6.07     11.41     4.01  6.45    5.02  11.11    5.96   

     page_blocks  glass  
WC          1.15  14.17  
KNN         3.68  17.82  
DT          0.

In [37]:
print('Tab oversampling method: DBSMOTE' )
print(pd.DataFrame.from_dict(dico_over_med['DBSMOTE']))
print(pd.DataFrame.from_dict(dico_over_std['DBSMOTE']))

Tab oversampling method: DBSMOTE
     wisconsin  yeast4  vehicle1  Subcl35    Paw  ecoli1   pima  yeast6  \
WC       95.14   63.37     62.82    68.91  75.16   84.40  64.02   77.97   
KNN      96.10   78.66     61.24    77.59  85.59   86.90  73.81   88.45   
DT       94.29   63.70     67.87    69.97  72.91   83.04  68.16   73.96   
SVM      96.48   74.39     64.72    50.00  53.77   85.51  55.27   89.97   

     page_blocks  glass  
WC         92.29  67.50  
KNN        83.33  75.00  
DT         99.44  63.38  
SVM        95.76  50.00  
     wisconsin  yeast4  vehicle1  Subcl35    Paw  ecoli1  pima  yeast6  \
WC        2.41   10.83      5.10     8.32   6.32   11.97  6.49   11.49   
KNN       2.66    8.56      3.37     4.10   5.26    5.08  4.05    9.78   
DT        2.15    6.89      5.38     5.99   6.06    5.81  2.77    9.79   
SVM       2.57    7.73     19.86     4.72  11.06    6.58  9.59   12.94   

     page_blocks  glass  
WC         11.80  18.95  
KNN        15.00  17.62  
DT         1

In [38]:
print('Tab oversampling method: GSSMOTE' )
print(pd.DataFrame.from_dict(dico_over_med['GSSMOTE']))
print(pd.DataFrame.from_dict(dico_over_std['GSSMOTE']))

Tab oversampling method: GSSMOTE
     wisconsin  yeast4  vehicle1  Subcl35    Paw  ecoli1   pima  yeast6  \
WC       95.47   74.95     63.89    70.71  76.08   82.75  64.65   80.84   
KNN      97.59   85.79     71.71    78.68  85.33   90.68  72.02   88.14   
DT       94.38   65.61     68.17    73.22  75.30   85.16  68.84   74.32   
SVM      96.71   82.17     59.81    50.26  53.33   86.12  54.18   87.97   

     page_blocks  glass  
WC         93.23  65.99  
KNN        95.45  79.21  
DT         94.43  58.26  
SVM        95.56  50.00  
     wisconsin  yeast4  vehicle1  Subcl35   Paw  ecoli1  pima  yeast6  \
WC        3.07   11.22      4.45     4.46  5.75    7.45  4.58    5.23   
KNN       1.66    6.49      5.46     4.74  2.65    4.03  3.97    3.27   
DT        3.95   11.94      4.18     7.71  8.65    6.40  2.50   13.18   
SVM       2.21    8.77     15.59     9.44  9.17    4.56  8.53   11.12   

     page_blocks  glass  
WC          9.56  26.17  
KNN         4.27  11.34  
DT         11.14 

In [39]:
print('Tab oversampling method: WSSMOTE' )
print(pd.DataFrame.from_dict(dico_over_med['WSSMOTE']))
print(pd.DataFrame.from_dict(dico_over_std['WSSMOTE']))

Tab oversampling method: WSSMOTE
     wisconsin  yeast4  vehicle1  Subcl35    Paw  ecoli1   pima  yeast6  \
WC       95.95   67.74     64.14    67.00  74.40   85.53  64.15   77.57   
KNN      98.04   87.55     71.80    76.98  86.23   91.76  70.56   88.92   
DT       94.04   66.12     70.26    69.32  73.38   85.48  67.23   74.15   
SVM      96.96   73.60     75.09    50.00  50.00   88.58  59.35   86.64   

     page_blocks  glass  
WC         92.57  64.08  
KNN        97.69  80.00  
DT        100.00  58.26  
SVM        95.55  54.69  
     wisconsin  yeast4  vehicle1  Subcl35   Paw  ecoli1  pima  yeast6  \
WC        1.92    8.73      4.02     9.47  7.90    9.54  5.06    8.23   
KNN       1.52    5.54      5.26     4.87  4.41    4.11  3.83    6.81   
DT        2.25    6.03      4.53     6.05  7.87    7.08  4.92   11.74   
SVM       1.74    4.52     11.65     8.89  3.68    4.95  6.11    8.27   

     page_blocks  glass  
WC         11.60   9.38  
KNN         3.77   8.59  
DT          0.56 